In [1]:
!pip install IPython
!pip install rich
!pip install plotly.express
!pip install dash
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sqlite3
import pandas as pd
import numpy as np
from numpy import *
import IPython.display
from IPython.display import display
from IPython.display import Image
from IPython.display import SVG
import rich
from rich.traceback import install
from rich import print
from rich.console import Console
from rich.syntax import Syntax
from rich.markdown import Markdown
import seaborn as sns
import matplotlib.pyplot as plt
from rich.panel import Panel
from rich.text import Text
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
import time
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

conn = sqlite3.connect("Metacritic2020-202211.db")
install(show_locals=True)
console = Console()
IPython.display.clear_output()

In [2]:
line=None
def game_check(conn, line):
    platform = []
    date = []
    critic = []
    user_score = []
    user_number = []
    user_score_good = []
    user_score_medium = []
    user_score_bad = []
    
    def gamegrade(conn, line):
        """
        Find the game grade
        If critic_value_all is empty, return average of media review ratings
        """
        n = 0
        line = line.strip()
        with conn:
            sqlcritic= "SELECT critic_value_all FROM game_list INNER JOIN game_index USING(G_ID) WHERE game_name = ?"
            cur = conn.cursor()
            cur.execute(sqlcritic, [line])
            result_critic = cur.fetchall()
            for result in result_critic:
                try:
                    new_result = int(result[0])
                    critic.append(new_result)
                    n = n + 1
                except:
                    sqlgid = "SELECT G_ID FROM game_list INNER JOIN game_index USING(G_ID) WHERE game_name = ?"
                    cur.execute(sqlgid, [line])
                    result_gid = cur.fetchall()[n]
                    new_unll_gid = result_gid[0]
                    sqlavg = "SELECT AVG(critic_grade) FROM media_views WHERE G_ID=?"
                    cur.execute(sqlavg, [new_unll_gid])
                    result_new_avg = cur.fetchone()
                    critic.append(result_new_avg[0])
                    n = n + 1

    def gameplatform(conn, line):
        """
        Find the game's release platform
        """
        line = line.strip()
        with conn:
            sqlplatform= "SELECT game_platform FROM game_index WHERE game_name = ?"
            cur = conn.cursor()
            cur.execute(sqlplatform, [line])
            result_platform = cur.fetchall()
            for result in result_platform:
                platform.append(str(result[0]))


    def gamedate(conn, line):
        """
        Find the game's release date
        """
        line = line.strip()
        with conn:
            sqldate= "SELECT game_date FROM game_index INNER JOIN game_list USING(G_ID) WHERE game_name = ?"
            cur = conn.cursor()
            cur.execute(sqldate, [line])
            result_date = cur.fetchall()
            for result in result_date:
                date.append(str(result[0]))


    def gameuserscore(conn, line):
        """
        Find the game's user score
        """
        line = line.strip()
        with conn:
            sqluserscore= "SELECT user_score FROM game_index INNER JOIN user_ratings USING(G_ID) WHERE game_name = ?"
            cur = conn.cursor()
            cur.execute(sqluserscore, [line])
            result_userscore = cur.fetchall()
            for result in result_userscore:
                user_score.append(float(result[0]))

    def gameusernum(conn, line):
        """
        Find the game's number of user ratings
        """
        line = line.strip()
        with conn:
            sqlusernum= "SELECT user_numbers FROM user_ratings INNER JOIN game_index USING(G_ID) WHERE game_name = ?"
            cur = conn.cursor()
            cur.execute(sqlusernum, [line])
            result_usernum = cur.fetchall()
            for result in result_usernum:
                user_number.append(str(result[0]))

    def gameuserpositive(conn, line):
        """
        Find the game's good user ratings
        """
        line = line.strip()
        with conn:
            sqluserpositive= "SELECT user_positive_reviews FROM user_ratings INNER JOIN game_index USING(G_ID) WHERE game_name = ?"
            cur = conn.cursor()
            cur.execute(sqluserpositive, [line])
            result_userpos = cur.fetchall()
            for result in result_userpos:
                user_score_good.append(str(result[0]))

    def gameusermedium(conn, line):
        """
        Find the game's medium user ratings
        """
        line = line.strip()
        with conn:
            sqlusermedium= "SELECT user_mixed_reviews FROM user_ratings INNER JOIN game_index USING(G_ID) WHERE game_name = ?"
            cur = conn.cursor()
            cur.execute(sqlusermedium, [line])
            result_usernum = cur.fetchall()
            for result in result_usernum:
                user_score_medium.append(str(result[0]))

    def gameuserbad(conn, line):
        """
        Find the game's bad user ratings
        """
        line = line.strip()
        with conn:
            sqluserbad= "SELECT user_negative_reviews FROM user_ratings INNER JOIN game_index USING(G_ID) WHERE game_name = ?"
            cur = conn.cursor()
            cur.execute(sqluserbad, [line])
            result_usernum = cur.fetchall()
            for result in result_usernum:
                user_score_bad.append(str(result[0]))

    def gamesqlcheck(conn, line):
        """
        Main query part, using fuzzy check
        """        
        fuzzygamename = console.input("Please input a [bold yellow]Game Title[/]: ")
        fuzzygamename_index = []

        def gamenamefuzzy(fuzzygamename,conn):
            fuzzygamename = fuzzygamename.strip()
            with conn:
                cur = conn.cursor()
                cur.execute("SELECT game_name FROM game_index WHERE game_name LIKE ?", ('%' + fuzzygamename + '%',))
                result_usernum = cur.fetchall()
                for result in result_usernum:
                    fuzzygamename_index.append(str(result[0]))
            #print(fuzzygamename_index)
            list(set(fuzzygamename_index))
            n = 1
            d = dict()
            new_name_index = []
            # Create a new array to contain all the options
            for gamename in fuzzygamename_index:
                    if gamename not in new_name_index:
                        new_name_index.append(gamename)
            for name in new_name_index:
                print("[",n,"] :",name)
                n = n + 1
            try:
                input_num = int(input("Please Input the index number before these game names: ")) 
                if input_num <= len(new_name_index):
                    nameoutput = new_name_index[input_num-1]
                else:
                    print("Please Try again! ")
                    time.sleep(2)
                    return gamenamefuzzy(fuzzygamename, conn)
            except: 
                print("Please Try again! ")
                time.sleep(2) 
                return gamenamefuzzy(fuzzygamename, conn)
            return nameoutput

        line = gamenamefuzzy(fuzzygamename,conn)
        line = line.strip()
        with conn:
            sqlcheck= "SELECT * FROM game_index WHERE game_name=?"
            cur = conn.cursor()
            cur.execute(sqlcheck, [line])
            result = cur.fetchone()
            if result == None:
                print("No such game_name")
            else:
                gameplatform(conn, line)
                gamedate(conn, line)
                gamegrade(conn, line)
                gameuserscore(conn, line)
                gameusernum(conn, line)
                gameuserpositive(conn, line)
                gameusermedium(conn, line)
                gameuserbad(conn, line)
                GameDataStore = pd.DataFrame({'Platforms': platform,
                                              'Release Date': date,
                                              'Critic Score': critic,
                                              'User Score': user_score,
                                              'User Numbers': user_number})


                GraphUserScore = pd.DataFrame({'Platforms': platform,
                                               'Positive': user_score_good,
                                               'Medium': user_score_medium,
                                               'Bad': user_score_bad,
                                               })

                format_dict = {
                    'Critic Score': '{0:.1f}',
                    'User Score': '{0:.1f}',
                    'Release Date': lambda x: "{}".format(x.strftime('%Y-%m-%d')),
                }

                GameDataStore['Release Date'] = pd.to_datetime(GameDataStore['Release Date'], format='%Y-%m-%d')
                GameDataStore['Critic Score'] = pd.to_numeric(GameDataStore['Critic Score'], errors='coerce')
                GameDataStore['User Score'] = pd.to_numeric(GameDataStore['User Score'], errors='coerce')
                GraphUserScore['Positive'] = pd.to_numeric(GraphUserScore['Positive'], errors='coerce')
                GraphUserScore['Medium'] = pd.to_numeric(GraphUserScore['Medium'], errors='coerce')
                GraphUserScore['Bad'] = pd.to_numeric(GraphUserScore['Bad'], errors='coerce')

                def infoprint(conn, line):
                    """
                    Print basic information of subject
                    """
                    line = line.strip()
                    with conn:
                        
                        sqlgamesummary= "SELECT game_summary FROM game_details INNER JOIN game_index USING(G_ID) WHERE game_name = ?"
                        cur = conn.cursor()
                        cur.execute(sqlgamesummary, [line])
                        result_summary = str(cur.fetchone()[0])
                        result_summary_print = str("Basic summary:")
                        
                        sqlgamegenre= "SELECT game_genres FROM game_list INNER JOIN game_index USING(G_ID) WHERE game_name = ?"
                        cur = conn.cursor()
                        cur.execute(sqlgamegenre, [line])
                        result_genre = str(cur.fetchone()[0])
                        
                        sqlgamesummary= "SELECT game_level FROM game_list INNER JOIN game_index USING(G_ID) WHERE game_name = ?"
                        cur = conn.cursor()
                        cur.execute(sqlgamesummary, [line])
                        result_level = str(cur.fetchone()[0])
                        result_level_print = str("Rated:")
                        
                        IPython.display.clear_output()
                        critic_info = float(GameDataStore['Critic Score'].mean())
                        platform_info_str = str("This game can be played on: ")
                        critic_info_str = str("The average critic score of this game on all platforms: ")
                        user_info = float(GameDataStore['User Score'].mean())
                        user_info_str = str("The average user score of this game on all platforms: ")
                        
                        console = Console()
                        console.print(line, style="bold green")
                        console.print(result_level_print,result_level)
                        console.print(result_genre)
                        console.print(f'[bold]{result_summary_print}[/]','\n',result_summary)
                        # Output text string styles
                        console.print(platform_info_str,list(GameDataStore['Platforms']))
                        console.print(critic_info_str, f'[green]{("{:.1f}".format(critic_info))}[/]')
                        console.print(user_info_str, f'[green]{("{:.1f}".format(user_info))}[/]')
                infoprint(conn, line)
                """
                Create Styled Pandas Dataframe
                """
                console.print("Performance by platform(s)")
                display(GameDataStore.sort_values(by=['Critic Score'],axis=0, ascending=False, inplace=False, na_position='last')
                            .reset_index(drop=True).set_index(GameDataStore.index+1)
                        .style
                        .highlight_between(left=7.5, right=10, subset=['User Score'],
                                           props='color:#2E7D32;font-weight:bold')
                        .highlight_between(left=4.0, right=7.5, subset=['User Score'],
                                           props='color:#FFA000;font-weight:bold')
                        .highlight_between(left=0.0, right=3.9, subset=['User Score'],
                                           props='color:#EF5350;font-weight:bold')
                        .highlight_between(left=80.0, right=100.0, subset=['Critic Score'],
                                           props='color:white;background-color:#2E7D32;font-weight:bold')
                        .highlight_between(left=40.0, right=79.9, subset=['Critic Score'],
                                           props='color:white;background-color:#FFA000;font-weight:bold')
                        .highlight_between(left=0.0, right=39.9, subset=['Critic Score'],
                                           props='color:white;background-color:#EF5350;font-weight:bold')
                        .set_properties(**{'border-color': 'white'})
                        .format(format_dict, na_rep=' -TBD- '))

                """
                Create barcharts of User Ratings' distribution
                """
                figuser = go.Figure(data=[
                                go.Bar(y=GraphUserScore['Platforms'],x=GraphUserScore['Positive'],name="Positive",text=GraphUserScore['Positive'],marker_color='rgb(102,204,51)',orientation='h'), 
                                go.Bar(y=GraphUserScore['Platforms'],x=GraphUserScore['Medium'],name="Mixed",text=GraphUserScore['Medium'],marker_color='rgb(255,204,51)',orientation='h'),
                                go.Bar(y=GraphUserScore['Platforms']
                                       ,x=GraphUserScore['Bad'],name="Bad",text=GraphUserScore['Bad'],marker_color='rgb(255,0,0)',orientation='h'),
                                ])
                figuser.update_layout(title = 'Distribution of '+line+"'s User Ratings",barmode="stack",yaxis={'categoryorder':'total ascending'})
                figuser.update_traces(textposition="inside")
                figuser.show()          
                
    gamesqlcheck(conn, line)

In [3]:
line = None
def developersql(conn,line):    
        title = []
        platform = []
        date = []
        critic = []
        user_score = []
        user_number = []
        temp = []
        user_score_good = []
        user_score_medium = []
        user_score_bad = []

        console = Console()
        line=None

        def gamename(conn, line):
            """
            Find the game name
            """
            line = line.strip()
            with conn:
                sqlname= "SELECT game_name FROM game_index INNER JOIN game_list USING(G_ID) WHERE game_developer = ?"
                cur = conn.cursor()
                cur.execute(sqlname, [line])
                result_name = cur.fetchall()
                for result in result_name:
                    title.append(str(result[0]))


        def gameplatform(conn, line):
            """
            Find the game platform
            """
            line = line.strip()
            with conn:
                sqlplatform= "SELECT game_platform FROM game_index INNER JOIN game_list USING(G_ID) WHERE game_developer = ?"
                cur = conn.cursor()
                cur.execute(sqlplatform, [line])
                result_platform = cur.fetchall()
                for result in result_platform:
                    platform.append(str(result[0]))


        def gamedate(conn, line):
            """
            Find the game release date
            """
            line = line.strip()
            with conn:
                sqldate= "SELECT game_date FROM game_index INNER JOIN game_list USING(G_ID) WHERE game_developer = ?"
                cur = conn.cursor()
                cur.execute(sqldate, [line])
                result_date = cur.fetchall()
                for result in result_date:
                    date.append(str(result[0]))


        def gameuserscore(conn, line):
            """
            Find the game user score
            """
            line = line.strip()
            with conn:
                sqluserscore= "SELECT user_score FROM user_ratings INNER JOIN game_list USING(G_ID) WHERE game_developer = ?"
                cur = conn.cursor()
                cur.execute(sqluserscore, [line])
                result_userscore = cur.fetchall()
                for result in result_userscore:
                    user_score.append(str(result[0]))


        def gameusernum(conn, line):
            """
            Find the game user ratings
            """
            line = line.strip()
            with conn:
                sqlusernum= "SELECT user_numbers FROM user_ratings INNER JOIN game_list USING(G_ID) WHERE game_developer = ?"
                cur = conn.cursor()
                cur.execute(sqlusernum, [line])
                result_usernum = cur.fetchall()
                for result in result_usernum:
                    user_number.append(str(result[0]))

        def gamegrade(conn, line):
            """
            Find the game grade
            """
            n = 0
            line = line.strip()
            with conn:
                sqlcritic= "SELECT critic_value_all FROM game_index INNER JOIN game_list USING(G_ID) WHERE game_developer = ?"
                sqlgid = "SELECT G_ID FROM game_list WHERE game_developer = ? "
                cur = conn.cursor()
                cur.execute(sqlcritic, [line])
                result_critic = cur.fetchall()
                for result in result_critic:
                    #尝试将none值设为整数格式，如果不是，则跳入except
                    try:
                        new_result = int(result[0])
                        critic.append(new_result)
                        #计数，让数组位置与none的位置平行
                        n = n + 1
                    except:
                        sqlgid = "SELECT G_ID FROM game_list WHERE game_developer = ? "
                        cur.execute(sqlgid, [line])
                        # 取出第 n 个 G_ID 让后续与之对应
                        result_gid = cur.fetchall()[n]
                        new_unll_gid = result_gid[0]
                        #print(new_unll_gid)
                        sqlavg = "SELECT AVG(critic_grade) FROM media_views WHERE G_ID=?"
                        cur.execute(sqlavg, [new_unll_gid])
                        #print(new_unll_gid)
                        result_new_avg = cur.fetchone()
                        critic.append(result_new_avg[0])
                        # 计数，让数组位置与none的位置平行
                        n = n + 1          

        def gameuserpositive(conn, line):
            """
            Find the game user ratings
            """
            line = line.strip()
            with conn:
                sqluserpositive= "SELECT user_positive_reviews FROM user_ratings INNER JOIN game_list USING(G_ID) WHERE game_developer = ?"
                cur = conn.cursor()
                cur.execute(sqluserpositive, [line])
                result_userpos = cur.fetchall()
                for result in result_userpos:
                    user_score_good.append(str(result[0]))

        def gameusermedium(conn, line):
            """
            Find the game user ratings
            """
            line = line.strip()
            with conn:
                sqlusermedium= "SELECT user_mixed_reviews FROM user_ratings INNER JOIN game_list USING(G_ID) WHERE game_developer = ?"
                cur = conn.cursor()
                cur.execute(sqlusermedium, [line])
                result_usernum = cur.fetchall()
                for result in result_usernum:
                    user_score_medium.append(str(result[0]))

        def gameuserbad(conn, line):
            """
            Find the game user ratings
            """
            line = line.strip()
            with conn:
                sqluserbad= "SELECT user_negative_reviews FROM user_ratings INNER JOIN game_list USING(G_ID) WHERE game_developer = ?"
                cur = conn.cursor()
                cur.execute(sqluserbad, [line])
                result_usernum = cur.fetchall()
                for result in result_usernum:
                    user_score_bad.append(str(result[0]))        

        def developersqlcheck(conn, line): 
            """
            Main check
            """
            fuzzydevelopername = console.input("Please input [bold yellow]Game Developer[/]'s name: ")
            fuzzydevelopername_index = []

            def developernamefuzzy(fuzzydevelopername,conn):
                fuzzydevelopername = fuzzydevelopername.strip()
                with conn:
                    cur = conn.cursor()
                    cur.execute("SELECT game_developer FROM game_list WHERE game_developer LIKE ?", ('%' + fuzzydevelopername + '%',))
                    result_usernum = cur.fetchall()
                    for result in result_usernum:
                        fuzzydevelopername_index.append(str(result[0]))
                #print(fuzzydevelopername_index)
                list(set(fuzzydevelopername_index))
                n = 1
                d = dict()
                new_name_index = []
                #新建数组 将所查到的名称按次序放入
                for gamename in fuzzydevelopername_index:
                        if gamename not in new_name_index:
                            new_name_index.append(gamename)
                #print(new_name_index)
                for name in new_name_index:
                    print("[",n,"] :",name)
                    n = n + 1
                try:
                    input_num = int(console.input("Please Input the [bold yellow]index number[/] before these developer's names: "))
                    if input_num <= len(new_name_index):
                        nameoutput = new_name_index[input_num-1]
                    else:
                        print("Please Try Again!")
                        time.sleep(1)
                        return developernamefuzzy(fuzzydevelopername,conn)
                except:
                    print("Please Try Again! ")
                    time.sleep(1)
                    return developernamefuzzy(fuzzydevelopername, conn)
                return nameoutput

            line = developernamefuzzy(fuzzydevelopername,conn)            
            line = line.strip()
            with conn:
                sqlcheck= "SELECT * FROM game_list WHERE game_developer=?"
                cur = conn.cursor()
                cur.execute(sqlcheck, [line])
                result = cur.fetchone()
                if result == None:
                    print("No such developer")
                else:
                    gamename(conn, line)
                    gameplatform(conn, line)
                    gamedate(conn, line)
                    gameuserscore(conn, line)
                    gameusernum(conn, line)
                    gamegrade(conn, line)
                    gameuserpositive(conn, line)
                    gameusermedium(conn, line)
                    gameuserbad(conn, line)
                    DeveloperDataStore = pd.DataFrame({'Game Title': title,
                                                       'Platforms': platform,
                                                       'Release Date': date,
                                                       'Critic Score': critic,
                                                       'User Score': user_score,
                                                       'User Number': user_number,}
                                                       )

                    GraphUserScore = pd.DataFrame({'Game Title': title,
                                                   'Game Platform':platform,
                                                       'Positive': user_score_good,
                                                       'Medium': user_score_medium,
                                                       'Bad': user_score_bad,
                                                       }) 



                    format_dict = { 
                       'Critic Score': '{0:.1f}', 
                       'User Score': '{0:.1f}', 
                       'Release Date':lambda x: "{}".format(x.strftime('%Y-%m-%d')),
                        }

                    DeveloperDataStore['Release Date'] = pd.to_datetime(DeveloperDataStore['Release Date'], format='%Y-%m-%d')
                    DeveloperDataStore['Critic Score'] = pd.to_numeric(DeveloperDataStore['Critic Score'],errors='coerce')
                    DeveloperDataStore['User Score'] = pd.to_numeric(DeveloperDataStore['User Score'],errors='coerce')
                    GraphUserScore['Positive'] = pd.to_numeric(GraphUserScore['Positive'],errors='coerce')
                    GraphUserScore['Medium'] = pd.to_numeric(GraphUserScore['Medium'],errors='coerce')
                    GraphUserScore['Bad'] = pd.to_numeric(GraphUserScore['Bad'],errors='coerce')

                    def infoprint(conn,line):
                        line = line.strip()
                        with conn:
                            result_info = float(DeveloperDataStore['Critic Score'].mean())
                            result_info_str = str("The average critic score of this developer:")
                            console = Console()
                            console.print(line,style="bold green")
                            #Output text string styles
                            console.print(result_info_str,f'[green]{("{:.1f}".format(result_info))}[/]')
                    IPython.display.clear_output()
                    infoprint(conn,line)
                    print("Performance by games")
                    display(DeveloperDataStore
                            .sort_values(by=['Critic Score'],axis=0, ascending=False, inplace=False, na_position='last')
                            .reset_index(drop=True).set_index(DeveloperDataStore.index+1)
                            .style
                            .highlight_between(left=7.5,right=10,subset=['User Score'],props='color:rgb(102,204,51);font-weight:bold')
                            .highlight_between(left=4.0,right=7.5,subset=['User Score'],props='color:rgb(255,204,51);font-weight:bold')
                            .highlight_between(left=0.0,right=3.9,subset=['User Score'],props='color:rgb(255,0,0);font-weight:bold')
                            .highlight_between(left=80.0,right=100.0,subset=['Critic Score'],props='color:white;background-color:rgb(102,204,51);font-weight:bold')
                            .highlight_between(left=40.0,right=79.9,subset=['Critic Score'],props='color:white;background-color:rgb(255,204,51);font-weight:bold')
                            .highlight_between(left=0.0,right=39.9,subset=['Critic Score'],props='color:white;background-color:rgb(255,0,0);font-weight:bold')
                            .format(format_dict,na_rep=' -TBD- '))
                    
                    # pie charts
                    Criticgrade = pd.DataFrame(critic)
                    Criticgrade.columns=["critic"]

                    Low = []
                    Middle = []
                    High = []

                    criticgrade = map(float,DeveloperDataStore['Critic Score'])

                    criticgrade = criticgrade
                    for item in criticgrade:
                        if item < 40:
                            Low.append(item)
                        elif item >= 40 and item < 80:
                            Middle.append(item)
                        elif item > 80:
                            High.append(item)

                    CriticGrade = {"Bad" : Low, "Medium": Middle, "Good": High}

                    #pd.DataFrame(CriticGrade)
                    c=pd.DataFrame(dict([(k,pd.Series(v)) for k,v in CriticGrade.items()]))
                    c.describe()

                    grade = c.describe().loc[['count'],['Bad','Medium','Good']]
                    grade1 = grade.T
                    degree = ['Bad → <=40','Medium → 40 ~ 80','Good → >=80']
                    grade1.insert(0,'Degree',degree)
                    grade1

                    figpie = px.pie(grade1, values='count', names='Degree', hole=0.3,title=line+"'s Media Evaluation Distribution",color="Degree",
                                    color_discrete_map={'Bad → <=40':'rgb(255,0,0)',
                                                         'Medium → 40 ~ 80':'rgb(255,204,51)',
                                                         'Good → >=80':'rgb(102,204,51)'})
                    figpie.show()
                    
                    # bar charts
                    figuser = go.Figure(data=[
                                    go.Bar(y=GraphUserScore['Game Title']+" - "+GraphUserScore['Game Platform'],x=GraphUserScore['Positive'],name="Positive",text=GraphUserScore['Positive'],marker_color='rgb(102,204,51)',orientation='h'), 
                                    go.Bar(y=GraphUserScore['Game Title']+" - "+GraphUserScore['Game Platform'],x=GraphUserScore['Medium'],name="Mixed",text=GraphUserScore['Medium'],marker_color='rgb(255,204,51)',orientation='h'),
                                    go.Bar(y=GraphUserScore['Game Title']+" - "+GraphUserScore['Game Platform'],x=GraphUserScore['Bad'],name="Bad",text=GraphUserScore['Bad'],marker_color='rgb(255,0,0)',orientation='h'),
                                    ])

                     # change the bar mode：更新柱状图的mode
                    figuser.update_layout(title = 'Distribution of '+line+"'s User Ratings",barmode="stack",yaxis={'categoryorder':'total ascending'})
                    figuser.update_traces(textposition="inside")
                    figuser.show()                   
        developersqlcheck(conn, line)                                    

In [4]:
line=None 
def mediasql(conn,line):
    # Query specific media information
        cur = conn.cursor()
        game_name = []
        game_platform = []
        critic_grade = []
        critic_bodys = []
        
        def gamename(conn, line):

            line = line.strip()
            with conn:
                sqlname= "SELECT game_name FROM game_index INNER JOIN media_views USING(G_ID) WHERE CRITIC_VALUE = ?"
                cur = conn.cursor()
                cur.execute(sqlname, [line])
                result_name = cur.fetchall()
                for result in result_name:
                    game_name.append(str(result[0]))

        def gameplatform(conn, line):

            line = line.strip()
            with conn:
                sqlplatform= "SELECT game_platform FROM game_index INNER JOIN media_views USING(G_ID) WHERE CRITIC_VALUE = ?"
                cur = conn.cursor()
                cur.execute(sqlplatform, [line])
                result_platform = cur.fetchall()
                for result in result_platform:
                    game_platform.append(str(result[0]))

        def criticgrade(conn, line):
            line = line.strip()
            with conn:
                sqlgrade= "SELECT critic_grade FROM game_list INNER JOIN media_views WHERE CRITIC_VALUE = ?"
                cur = conn.cursor()
                cur.execute(sqlgrade, [line])
                result_grade = cur.fetchall()
                for result in result_grade:
                    critic_grade.append(str(result[0]))

        def criticbodys(conn, line):
            line = line.strip()
            with conn:
                sqlbodys= "SELECT critic_bodys FROM game_list INNER JOIN media_views WHERE CRITIC_VALUE = ?"       
                cur = conn.cursor()
                cur.execute(sqlbodys, [line])
                result_bodys = cur.fetchall()
                for result in result_bodys:
                    critic_bodys.append(str(result[0]))


        def mediasqlcheck(conn, line):
            fuzzymedianame = console.input("Please input [bold yellow]Media[/]'s name: ")
            fuzzymedianame_index = []
            def medianamefuzzy(fuzzymedianame,conn):
                fuzzymedianame = fuzzymedianame.strip()
                with conn:
                    cur = conn.cursor()
                    cur.execute("SELECT critic_value FROM media_views WHERE critic_value LIKE ?", ('%' + fuzzymedianame + '%',))
                    result_usernum = cur.fetchall()
                    for result in result_usernum:
                        fuzzymedianame_index.append(str(result[0]))
                #print(fuzzymedianame_index)
                list(set(fuzzymedianame_index))
                n = 1
                d = dict()
                new_name_index = []
                #新建数组 将所查到的名称按次序放入
                for gamename in fuzzymedianame_index:
                        if gamename not in new_name_index:
                            new_name_index.append(gamename)
                #print(new_name_index)
                for name in new_name_index:
                    print("[",n,"] :",name)
                    n = n + 1
                try:
                    input_num = int(console.input("Please Input the [bold yellow]index number[/] before these media's names: "))
                    if input_num <= len(new_name_index):
                        nameoutput = new_name_index[input_num-1]
                    else:
                        print("Please Try Again!")
                        time.sleep(1)
                        return medianamefuzzy(fuzzymedianame,conn)
                except:
                    print("Please Try Again! ")
                    time.sleep(1)
                    return medianamefuzzy(fuzzymedianame, conn)
                return nameoutput
            
            line = medianamefuzzy(fuzzymedianame,conn)            
            line = line.strip()
            with conn:
                sqlcheck= "SELECT * FROM media_views WHERE CRITIC_VALUE=?"
                cur = conn.cursor()
                cur.execute(sqlcheck, [line])
                checkresult = cur.fetchone()
                if checkresult == None:
                    print("No such media")
                else:
                    gamename(conn, line)
                    gameplatform(conn, line)
                    criticgrade(conn, line)
                    criticbodys(conn, line)
                    Mediainfo = {'Game Title': game_name,
                                'Platforms': game_platform,
                            'Critic Grade': critic_grade,
                        'Critic Bodys': critic_bodys}


                    format_dict = {'Critic Grade': '{0:.1f}'}


                    mediadisplay=pd.DataFrame(dict([(k,pd.Series(v)) for k,v in Mediainfo.items()]))
                    mediadisplay['Critic Grade'] = pd.to_numeric(mediadisplay['Critic Grade'],errors='coerce')
                    
                    def mediainfoprint(conn,line):
                        line = line.strip()
                        with conn:
                            result_info = float(mediadisplay['Critic Grade'].mean())
                            result_info_str = str("The average score given by this media:")
                            console = Console()
                            IPython.display.clear_output()
                            console.print(line,style="bold green")
                            #Output text string styles
                            console.print(result_info_str,f'[green]{("{:.1f}".format(result_info))}[/]')                                        
                    IPython.display.clear_output()
                    mediainfoprint(conn,line)
                    print("Media Critic Summary")
                    display(mediadisplay.reset_index(drop=True).set_index(mediadisplay.index+1)
                            .head(15).style
                                       .highlight_between(left=80.0,right=100.0,subset=['Critic Grade'],props='color:rgb(102,204,51);font-weight:bold')
                                       .highlight_between(left=40.0,right=79.9,subset=['Critic Grade'],props='color:rgb(255,204,51);font-weight:bold')
                                       .highlight_between(left=0,right=39.9,subset=['Critic Grade'],props='color:rgb(255,0,0);font-weight:bold')
                                       .format(format_dict,na_rep=' -TBD- '))
                    
                    Critic_grade_pie = pd.DataFrame(critic_grade)
                    Critic_grade_pie.columns=["Critic_grade"]

                    Low = []
                    Middle = []
                    High = []

                    critic_grade_pie = map(int,critic_grade)

                    critic_grade_pie = critic_grade_pie
                    for item in critic_grade_pie:
                        if item < 40:
                            Low.append(item)
                        elif item >= 40 and item < 80:
                            Middle.append(item)
                        elif item > 80:
                            High.append(item)

                    Critic_Grade_1 = {"Bad" : Low, "Medium": Middle, "Good": High}

                    #pd.DataFrame(CriticGrade)
                    c=pd.DataFrame(dict([(k,pd.Series(v)) for k,v in Critic_Grade_1.items()]))
                    c.describe()

                    grade = c.describe().loc[['count'],['Bad','Medium','Good']]
                    grade1 = grade.T
                    degree = ['Bad → <=40','Medium → 40 ~ 80','Good → >=80']
                    grade1.insert(0,'Degree',degree)
                    grade1

                    figpie = px.pie(grade1, values='count', names='Degree', hole=0.3,title=line+"'s Media Evaluation Distribution",color="Degree",
                                    color_discrete_map={'Bad → <=40':'rgb(255,0,0)',
                                                         'Medium → 40 ~ 80':'rgb(255,204,51)',
                                                         'Good → >=80':'rgb(102,204,51)'})
                    figpie.show()

                    # Import the words cloud of critic bodys
                    from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

                    for i in game_name:
                        STOPWORDS.add(i)

                    for p in game_platform:
                        STOPWORDS.add(p)

                    STOPWORDS.add("game")
                    STOPWORDS.add("one")
                    STOPWORDS.add("play")
                    STOPWORDS.add("one")
                    STOPWORDS.add("doesn_t")
                    STOPWORDS.add("lack")
                    STOPWORDS.add("lot")
                    STOPWORDS.add("doesn")
                    STOPWORDS.add("way")
                    STOPWORDS.add("end")
                    STOPWORDS.add("make")
                    STOPWORDS.add("isn")
                    STOPWORDS.add("come")
                    STOPWORDS.add("bit")
                    STOPWORDS.add("take")
                    STOPWORDS.add("let")
                    STOPWORDS.add("co")
                    STOPWORDS.add("op")
                    STOPWORDS.add("much")
                    STOPWORDS.add("fun")
                    STOPWORDS.add("time")
                    STOPWORDS.add("don")
                    STOPWORDS.add("know")
                    STOPWORDS.add("genre")
                    STOPWORDS.add("want")
                    STOPWORDS.add("title")
                    STOPWORDS.add("manage")
                    STOPWORDS.add("think")
                    STOPWORDS.add("offer")
                    STOPWORDS.add("want")


                    mw = ' '.join(text for text in critic_bodys)
                    mw_cloud = WordCloud(background_color="white").generate(mw)
                    plt.figure(figsize=(8, 4))
                    plt.imshow(mw_cloud, interpolation="bilinear")
                    plt.axis("off")
                    plt.title('Media wordcloud', size=24)
                    print(line,"main words:")
                    plt.show()
        mediasqlcheck(conn, line)

In [5]:
def alltimetrend(conn):
    title=[]
    platform=[]
    developer=[]
    date=[]
    date_year=[]
    critic_grade=[]
    user_grade=[]
    user_ratings=[]
    user_ratings_pos=[]
    
    #Query for all title
    def selecttitle(conn):    
        with conn:
            sqlname= "SELECT game_name FROM game_index,game_list WHERE game_index.G_ID=game_list.G_ID"
            cur = conn.cursor()
            cur.execute(sqlname)
            result_name = cur.fetchall()
            for result in result_name:
                title.append(str(result[0]))
    
    def selectplatform(conn):    
        with conn:
            sqlplatform= "SELECT game_platform FROM game_index,game_list WHERE game_index.G_ID=game_list.G_ID"
            cur = conn.cursor()
            cur.execute(sqlplatform)
            result_platform = cur.fetchall()
            for result in result_platform:
                platform.append(str(result[0]))
    
    def selectdeveloper(conn):    
        with conn:
            sqldeveloper= "SELECT game_developer FROM game_list,game_index WHERE game_index.G_ID=game_list.G_ID"
            cur = conn.cursor()
            cur.execute(sqldeveloper)
            result_developer = cur.fetchall()
            for result in result_developer:
                developer.append(str(result[0]))
                            
    def selectyear(conn):    
        with conn:
            sqldate= "SELECT game_date FROM game_list,game_index WHERE game_index.G_ID=game_list.G_ID"
            cur = conn.cursor()
            cur.execute(sqldate)
            result_release = cur.fetchall()
            for result in result_release:
                date_year.append(str(result[0])[0:4])
                
    
    def selectgrade(conn):
            n = 0
            with conn:
                sqlcritic= "SELECT critic_value_all FROM game_index,game_list WHERE game_index.G_ID=game_list.G_ID"
                cur = conn.cursor()
                cur.execute(sqlcritic)
                result_critic = cur.fetchall()
                for result in result_critic:
                    #尝试将none值设为整数格式，如果不是，则跳入except
                    try:
                        new_result = int(result[0])
                        critic_grade.append(new_result)
                        #计数，让数组位置与none的位置平行
                        n = n + 1
                    except:
                        sqlgid = "SELECT G_ID FROM game_list"
                        cur.execute(sqlgid)
                        # 取出第 n 个 G_ID 让后续与之对应
                        result_gid = cur.fetchall()[n]
                        new_unll_gid = result_gid[0]
                        #print(new_unll_gid)
                        sqlavg = "SELECT AVG(critic_grade) FROM media_views WHERE G_ID=?"
                        cur.execute(sqlavg, [new_unll_gid])
                        #print(new_unll_gid)
                        result_new_avg = cur.fetchone()
                        critic_grade.append(result_new_avg[0])
                        # 计数，让数组位置与none的位置平行
                        n = n + 1 
    
    def selectuserscore(conn):    
        with conn:
            sqluserscore= "SELECT user_score FROM user_ratings,game_index WHERE game_index.G_ID=user_ratings.G_ID"
            cur = conn.cursor()
            cur.execute(sqluserscore)
            result_userscore = cur.fetchall()
            for result in result_userscore:
                user_grade.append(str(result[0]))
                
    def selectusernum(conn):    
        with conn:
            sqlusernum= "SELECT user_positive_reviews+user_mixed_reviews+user_negative_reviews FROM user_ratings,game_index WHERE game_index.G_ID=user_ratings.G_ID"
            cur = conn.cursor()
            cur.execute(sqlusernum)
            result_usernum = cur.fetchall()
            for result in result_usernum:
                user_ratings.append(str(result[0]))
    
    def selectuserpos(conn):    
        with conn:
            sqluserpos= "SELECT user_positive_reviews FROM user_ratings,game_index WHERE game_index.G_ID=user_ratings.G_ID"
            cur = conn.cursor()
            cur.execute(sqluserpos)
            result_userpos = cur.fetchall()
            for result in result_userpos:
                user_ratings_pos.append(str(result[0]))
    
    
    selecttitle(conn)
    selectplatform(conn)
    selectdeveloper(conn)
    selectyear(conn)
    selectgrade(conn)
    selectuserscore(conn)
    selectusernum(conn)
    selectuserpos(conn)

    Allyeartrend = pd.DataFrame({'Game Title': title,
                                              'Platform': platform,
                                              'Developer': developer,
                                              'Release Year':date_year,
                                              'CriticScore':critic_grade,
                                              'UserScore': user_grade,
                                 })
    
    Allyeartrend['CriticScore'] = pd.to_numeric(Allyeartrend['CriticScore'],errors='coerce')
    Allyeartrend['UserScore'] = pd.to_numeric(Allyeartrend['UserScore'],errors='coerce')
    
    All2020 = pd.DataFrame(Allyeartrend[Allyeartrend["Release Year"].str.contains("2020")])
    All2021 = pd.DataFrame(Allyeartrend[Allyeartrend["Release Year"].str.contains("2021")])
    All2022 = pd.DataFrame(Allyeartrend[Allyeartrend["Release Year"].str.contains("2022")])
    
    AllPlatform2020Avg = All2020.groupby('Platform')['CriticScore','UserScore'].mean().reset_index().dropna(how='any')
    AllPlatform2020Avg.loc[0]=['All',AllPlatform2020Avg['CriticScore'].mean(),AllPlatform2020Avg['UserScore'].mean()]
    AllPlatform2020Avg = AllPlatform2020Avg.drop(AllPlatform2020Avg[AllPlatform2020Avg['Platform']=='Stadia'].index)
    AllPlatform2021Avg = All2021.groupby('Platform')['CriticScore','UserScore'].mean().reset_index().dropna(how='any')
    AllPlatform2021Avg.loc[0]=['All',AllPlatform2021Avg['CriticScore'].mean(),AllPlatform2021Avg['UserScore'].mean()]
    AllPlatform2021Avg = AllPlatform2021Avg.drop(AllPlatform2021Avg[AllPlatform2021Avg['Platform']=='Stadia'].index)
    AllPlatform2022Avg = All2022.groupby('Platform')['CriticScore','UserScore'].mean().reset_index().dropna(how='any')
    AllPlatform2022Avg.loc[6] = AllPlatform2022Avg.loc[5]
    AllPlatform2022Avg=AllPlatform2022Avg.shift(axis=0, periods=1)
    AllPlatform2022Avg.loc[0]=['All',AllPlatform2022Avg['CriticScore'].mean(),AllPlatform2022Avg['UserScore'].mean()]
    figall = go.Figure(data=[go.Bar(y=AllPlatform2020Avg['CriticScore'],x=AllPlatform2020Avg['Platform'],name="2020 CriticScore",
                                    marker_color='rgb(79, 195, 247)',opacity=0.5,text=AllPlatform2020Avg['CriticScore']), 
                              go.Bar(y=AllPlatform2021Avg['CriticScore'],x=AllPlatform2021Avg['Platform'],name="2021 CriticScore",
                                     marker_color='rgb(26, 35, 126)',opacity=0.5,text=AllPlatform2021Avg['CriticScore']),
                              go.Bar(y=AllPlatform2022Avg['CriticScore'],x=AllPlatform2022Avg['Platform'],name="2022 CriticScore",
                                     marker_color='rgb(49, 27, 146)',opacity=0.5,text=AllPlatform2022Avg['CriticScore']), 
                                    ])
    
     # change the bar mode：更新柱状图的mode
    figall.update_layout(title = '2020-2022 Overall Performance on both Critic and Player Scores',barmode="group")    
    figall.update_yaxes(range=[60,82])
    figall.update_traces(texttemplate='%{text:.4s}',textposition="outside")
    figall.add_trace(go.Scatter(x=AllPlatform2022Avg['Platform'], y=AllPlatform2022Avg['UserScore']*10,
                         mode='lines+markers',marker_color='rgb(233, 30, 99)',name='2022 UserScore',text=AllPlatform2022Avg['UserScore'],texttemplate='%{text:.3s}',textposition="top center"))
    figall.add_trace(go.Scatter(x=AllPlatform2021Avg['Platform'], y=AllPlatform2021Avg['UserScore']*10,
                         mode='lines+markers',marker_color='rgb(255, 112, 67)',name='2021 UserScore',text=AllPlatform2021Avg['UserScore'],texttemplate='%{text:.3s}',textposition="top center"))
    figall.add_trace(go.Scatter(x=AllPlatform2020Avg['Platform'], y=AllPlatform2020Avg['UserScore']*10,
                         mode='lines+markers',marker_color='rgb(255, 235, 59)',name='2020 UserScore',text=AllPlatform2020Avg['UserScore'],texttemplate='%{text:.3s}',textposition="top center"))
    figall.update_traces()
    figall.show()
        
    Low2020 = []
    Middle2020 = []
    High2020 = []
    
    Low2021 = []
    Middle2021 = []
    High2021 = []
    
    Low2022 = []
    Middle2022 = []
    High2022 = []

    criticgrade2020 = map(float,All2020['CriticScore'])
    criticgrade2021 = map(float,All2021['CriticScore'])
    criticgrade2022 = map(float,All2022['CriticScore'])

    criticgrade2020 = criticgrade2020
    for item in criticgrade2020:
        if item < 40:
            Low2020.append(item)
        elif item >= 40 and item < 80:
            Middle2020.append(item)
        elif item > 80:
            High2020.append(item)
    
    criticgrade2021 = criticgrade2021
    for item in criticgrade2021:
        if item < 40:
            Low2021.append(item)
        elif item >= 40 and item < 80:
            Middle2021.append(item)
        elif item > 80:
            High2021.append(item)
    
    criticgrade2022 = criticgrade2022
    for item in criticgrade2022:
        if item < 40:
            Low2022.append(item)
        elif item >= 40 and item < 80:
            Middle2022.append(item)
        elif item > 80:
            High2022.append(item)

    CriticGrade2020 = {"Bad" : Low2020, "Medium": Middle2020, "Good": High2020}
    CriticGrade2021 = {"Bad" : Low2021, "Medium": Middle2021, "Good": High2021}
    CriticGrade2022 = {"Bad" : Low2022, "Medium": Middle2022, "Good": High2022}

    #pd.DataFrame(CriticGrade)
    c2020=pd.DataFrame(dict([(k,pd.Series(v)) for k,v in CriticGrade2020.items()]))
    c2020.describe()
    
    c2021=pd.DataFrame(dict([(k,pd.Series(v)) for k,v in CriticGrade2021.items()]))
    c2021.describe()
    
    c2022=pd.DataFrame(dict([(k,pd.Series(v)) for k,v in CriticGrade2022.items()]))
    c2022.describe()

    grade2020 = c2020.describe().loc[['count'],['Bad','Medium','Good']]
    grade2020 = grade2020.T
    
    grade2021 = c2021.describe().loc[['count'],['Bad','Medium','Good']]
    grade2021 = grade2021.T
    
    grade2022 = c2022.describe().loc[['count'],['Bad','Medium','Good']]
    grade2022 = grade2022.T
    
    degree = ['Bad → <=40','Medium → 40 ~ 80','Good → >=80']
    
    grade2020.insert(0,'Degree',degree)
    grade2020
    
    grade2021.insert(0,'Degree',degree)
    grade2021
    
    grade2022.insert(0,'Degree',degree)
    grade2022
    
    piecolors=['rgb(255,0,0)','rgb(255,204,51)','rgb(102,204,51)']
    
    
    figpie = make_subplots(rows=1,cols=3,specs=[[{'type':'domain'}, {'type':'domain'},{'type':'domain'}]]
                          ,subplot_titles=['2020', '2021','2022'])
    figpie.add_trace(go.Pie(labels=degree,values=grade2020['count'],marker_colors=piecolors,name="2020"),1,1)
    figpie.add_trace(go.Pie(labels=degree,values=grade2021['count'],marker_colors=piecolors,name="2021"),1,2)
    figpie.add_trace(go.Pie(labels=degree,values=grade2022['count'],marker_colors=piecolors,name="2022"),1,3)
    figpie.update_traces(hole=.5,textposition='outside',hoverinfo="label+percent+name")
    figpie.update_layout(
    title_text="CriticScore Distribution")
    figpie.show()
    
    def piegenre(conn):
        game_genre = []
        game_level = []

        sqlgenre= "SELECT game_genres FROM game_list"
        cur = conn.cursor()
        cur.execute(sqlgenre)
        result_name = cur.fetchall()
        for result in result_name:
            game_genre.append(str(result[0]))

        gg = ' '.join(text for text in game_genre)
        gg1 = gg.split(",")
        gg2 = pd.DataFrame({'Genre' : gg1})
        gg2.insert(1,'Count',1)
        Gamegenre = gg2.groupby('Genre').count().reset_index()
        Gg = Gamegenre.head(5)

        sqllevel= "SELECT game_level FROM game_list"
        cur = conn.cursor()
        cur.execute(sqllevel)
        result_level = cur.fetchall()
        for result in result_level:
            game_level.append(str(result[0]))

        gl = pd.DataFrame({'Level' : game_level})
        gl.insert(1,'Count',1)
        Gamelevel = gl.groupby('Level').count().reset_index()
        Gl = Gamelevel
        Gl.drop(3,axis=0,inplace=True)

        figgenrepie = make_subplots(rows=1,cols=2,specs=[[{'type':'domain'}, {'type':'domain'}]],subplot_titles=['Genres','Ratings'])
        figgenrepie.add_trace(go.Pie(labels=Gg['Genre'],values=Gg['Count'],name="Genre",textinfo='label+percent',
                             insidetextorientation='radial'),1,1)
        figgenrepie.add_trace(go.Pie(labels=Gl['Level'],values=Gl['Count'],name="Level",textinfo='label+percent',
                             insidetextorientation='radial'),1,2)
        figgenrepie.update_traces(textposition="inside")
        figgenrepie.update_layout(title_text="Occurrence of Game Genres and Ratings 2020-2022")
        figgenrepie.show()
    
    piegenre(conn)
    
    #Top 10 Highest Score
    Allyearhighestcritic = Allyeartrend.sort_values(by='CriticScore',ascending=False).head(10)
    figtopcritic = go.Figure(data=[
                        go.Bar(y=Allyearhighestcritic['Game Title']+" - "+Allyearhighestcritic['Platform'],x=Allyearhighestcritic['CriticScore'],
                               marker_color='rgb(102,187,106)',text=Allyearhighestcritic['CriticScore'],orientation='h'), 
                        ])

                     # change the bar mode：更新柱状图的mode
    figtopcritic.update_layout(title = "Top 10 Highest Scores (by CriticScores)",yaxis={'categoryorder':'total ascending'})
    figtopcritic.update_traces(textposition="inside")
    figtopcritic.update_xaxes(range=[80,100])
    figtopcritic.show()
    
    #Top 10 Highest UserScore
    Allyearhighestuser = Allyeartrend.sort_values(by='UserScore',ascending=False).head(10)
    figtopuser = go.Figure(data=[
                        go.Bar(y=Allyearhighestuser['Game Title']+" - "+Allyearhighestuser['Platform'],x=Allyearhighestuser['UserScore'],
                               marker_color='rgb(9,187,159)',text=Allyearhighestuser['UserScore'],orientation='h'), 
                        ])

                     # change the bar mode：更新柱状图的mode
    figtopuser.update_layout(title = "Top 10 Highest Scores (by UserScores)",yaxis={'categoryorder':'total ascending'})
    figtopuser.update_traces(textposition="inside")
    figtopuser.update_xaxes(range=[8,10])
    figtopuser.show()
    
    #Top 10 Popular
    Allyearpopular = pd.DataFrame({'Game Title': title,
                                              'Platform': platform,
                                              'UserNumbers': user_ratings,
                                              'UserPositive':user_ratings_pos,})
    Allyearpopular['UserNumbers'] = pd.to_numeric(Allyearpopular['UserNumbers'],errors='coerce')
    Allyearpopular['UserPositive'] = pd.to_numeric(Allyearpopular['UserPositive'],errors='coerce')
    Allyearpopular = Allyearpopular.sort_values(by='UserNumbers',ascending=False).head(10)
    
    figpopular = go.Figure(data=[
                        go.Bar(y=Allyearpopular['Game Title']+" - "+Allyearpopular['Platform'],x=Allyearpopular['UserNumbers'],text=Allyearpopular['UserNumbers'],orientation='h'), 
                        ])

                     # change the bar mode：更新柱状图的mode
    figpopular.update_layout(title = "Top 10 Popular Games (by User Rating Number)",yaxis={'categoryorder':'total ascending'})
    figpopular.update_traces(textposition="inside")
    figpopular.show()
    
        #Top 10 Positive
    Allyeartrend["Minus"]=Allyeartrend[["CriticScore","UserScore"]].apply(lambda x:(x["CriticScore"]-10*x["UserScore"]),axis=1).abs()
    Allyearminus = Allyeartrend.sort_values(by='Minus',ascending=False).head(10)
    figtopminus = go.Figure(data=[
                        go.Bar(y= Allyearminus['Game Title']+" - "+ Allyearminus['Platform'],x= Allyearminus['Minus'],
                               marker_color='rgb(57,243,187)',text=Allyearminus['Minus'],orientation='h'), 
                        ])

                     # change the bar mode：更新柱状图的mode
    figtopminus.update_layout(title = "Top 10 with the biggest difference between media ratings and player experience",yaxis={'categoryorder':'total ascending'})
    figtopminus.update_traces(texttemplate='%{text:.4s}',textposition="inside")
    figtopminus.update_xaxes(range=[50,80])
    figtopminus.show()
    

In [6]:
import sqlite3
import time

from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pandas as pd

def Pt4_flatform(conn):
    #函数定义
    favourites = []
    attractives = []
    genres = []
    game_developer = []
    user_score = []
    critic_avg = []
    game_name = []
    every_critic_avg = []
    every_user_avg = []
    platform_name_index = ["PC", "PlayStation 4", "PlayStation 5", "Switch", "Xbox One", "Xbox Series X"]
    n = 1
    for platform_name in platform_name_index:
        print("[", n, "] :", platform_name)
        n = n + 1
    try:
        platform_input = int(input("Please input the No. Before the Platform: "))
    except:
        print("ERROR!!")
        time.sleep(1)
        return Pt4_flatform(conn)
    try:
        platform = platform_name_index[platform_input-1]
        print(platform)
    except:
        time.sleep(1)
        return Pt4_flatform(conn)
    print("[1] Query the Exact Year / Year-Month")
    print("[2] Query the period")
    choice = str(input("Please chose the query way："))

    def style_select(platform,choice,conn):
        if choice == "1":
    #特定年份/月份查询
            with conn:
                cur = conn.cursor()
                date = input("Please input the date:")
                cur.execute("SELECT * FROM game_index INNER JOIN game_list USING(G_ID) WHERE game_platform ='"+str(platform)+"' AND game_date LIKE ?", ('%' + date + '%',))
                result_platform = cur.fetchall()
                #返回这一年-这一个月的信息
                for result in result_platform:
                    critic_avg.append(every_avg_game(result[0], conn))
                    game_name.append(every_game_name(result[0], conn))
                    user_score.append(every_user_score(result[0], conn))
                    genres.append(find_genre(result[0], conn))
                    game_developer.append(result[7])
                    every_critic_avg.append(every_developer_critic_value_avg(result[7], conn))
                    every_user_avg.append(every_developer_critic_user_score_avg(result[7], conn))
                    attractives.append(user_attractives(result[0], conn))
                    favourites.append(user_favourites(result[0], conn))
                    
        elif choice == "2":
            with conn:
                cur = conn.cursor()
                time1 = input("Please input the beginning date: ")
                time2 = input("Please input the ending date: ")
                cur.execute("SELECT * FROM game_index INNER JOIN game_list USING(G_ID) WHERE game_platform ='"+str(platform)+"' AND game_date BETWEEN'"+str(time1)+"' AND '"+str(time2)+"'")
                result_platform = cur.fetchall()
                #返回这一年-这一个月的信息
                for result in result_platform:
                    critic_avg.append(every_avg_game(result[0], conn))
                    game_name.append(every_game_name(result[0], conn))
                    user_score.append(every_user_score(result[0], conn))
                    genres.append(find_genre(result[0], conn))
                    game_developer.append(result[7])
                    every_critic_avg.append(every_developer_critic_value_avg(result[7], conn))
                    every_user_avg.append(every_developer_critic_user_score_avg(result[7], conn))
                    attractives.append(user_attractives(result[0], conn))
                    favourites.append(user_favourites(result[0], conn))

        else:
            print("Wrong Input!! Please retype! ")
            time.sleep(1)
            return Pt4_flatform(conn)
    #输入特定的G_ID(result)，生成相应的avg填入critic_avg数组，
    def every_avg_game(line,conn):
        with conn:
            sql_critic_avg = "SELECT AVG(critic_grade) FROM media_views WHERE G_ID=?"
            cur = conn.cursor()
            cur.execute(sql_critic_avg, [line,])
            result_critic = cur.fetchone()
            every_avg = result_critic[0]
            return every_avg

    #进入相应的G_ID 返回相应的game_name
    def every_game_name(line,conn):
        with conn:
            sql_find_title = "SELECT game_name FROM game_index WHERE G_ID=?"
            cur = conn.cursor()
            cur.execute(sql_find_title, [line,])
            result_name = cur.fetchone()
            every_name = result_name[0]
            return str(every_name)

    def every_user_score(line,conn):
        with conn:
            sql_find_title = "SELECT user_score FROM user_ratings WHERE G_ID=?"
            cur = conn.cursor()
            cur.execute(sql_find_title, [line,])
            result_name = cur.fetchone()
            every_score = result_name[0]
            return str(every_score)


    #输入特定的developer，返回对应的 avg 分数
    def every_developer_critic_value_avg(line,conn):
        with conn:
            sql_find_developer_critic_value_avg = "SELECT AVG(critic_value_all) FROM game_list INNER JOIN game_index USING(G_ID) WHERE game_developer ='"+str(line)+"'"
            cur = conn.cursor()
            cur.execute(sql_find_developer_critic_value_avg)
            result_name = cur.fetchone()
            every_developer_critic_value_avg = result_name[0]
            return str(every_developer_critic_value_avg)

    #输入特定的developer，返回用户的 avg 分数
    def every_developer_critic_user_score_avg(line,conn):
        with conn:
            sql_find_developer_user_score_avg = "SELECT AVG(user_score) FROM game_list INNER JOIN user_ratings USING(G_ID) WHERE game_developer ='"+str(line)+"'"
            cur = conn.cursor()
            cur.execute(sql_find_developer_user_score_avg)
            result_name = cur.fetchone()
            every_developer_critic_user_score_avg = result_name[0]
            return str(every_developer_critic_user_score_avg)

    def find_genre(line,conn):
        with conn:
            sql_find_genre = "SELECT game_genres FROM game_list WHERE G_ID ='"+str(line)+"'"
            cur = conn.cursor()
            cur.execute(sql_find_genre)
            result_name = cur.fetchone()
            find_every_genre = result_name[0]
            return str(find_every_genre)

    def user_attractives(line,conn):
        with conn:
            sql_user_attrative = "SELECT user_positive_reviews+user_mixed_reviews+user_negative_reviews FROM user_ratings INNER JOIN game_index USING (G_ID) WHERE G_ID = '"+str(line)+"'"
            cur = conn.cursor()
            cur.execute(sql_user_attrative)
            result_name = cur.fetchone()
            find_every_attrative_score = result_name[0]
            return str(find_every_attrative_score)

    def user_favourites(line,conn):
        with conn:
            sql_user_favorite1 = " SELECT user_positive_reviews FROM user_ratings WHERE G_ID ='"+str(line)+"'"
            sql_user_favorite2 = " SELECT user_positive_reviews+user_mixed_reviews+user_negative_reviews FROM user_ratings WHERE G_ID ='" + str(line) + "'"
            cur = conn.cursor()
            cur.execute(sql_user_favorite1)
            result1 = cur.fetchone()
            cur.execute(sql_user_favorite2)
            result2 = cur.fetchone()
            try:
                result = str(round(int(result1[0]) / int(result2[0]),2))
            except:
                result = result2[0]
            return result

    def plot_wordcloud():
        from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
        STOPWORDS.add("Genre")
        STOPWORDS.add("Genre(S)")
        STOPWORDS.add("S")
        
        gg = ' '.join(text for text in genres)
        gg_cloud = WordCloud(background_color="white").generate(gg)
        plt.figure(figsize=(8, 4))
        plt.imshow(gg_cloud, interpolation="bilinear")
        plt.axis("off")
        plt.title('Game Genres', size=24)
        plt.show()

    #这一层启动运行
    style_select(platform, choice, conn)
    #写入 Data Frame Top 10 游戏 第 1、2个表
    format_dict = { 
                       'Critic Score': '{0:.1f}', 
                       'User Score': '{0:.1f}', 
                       'Popularity': '{0:.1f}',
                       'Favorable Rate': '{0:.1%}'
                        }

    Top10Game_user_score = pd.DataFrame({'Game Title': game_name,'User Score': user_score})
    Top10Game_critic_avg = pd.DataFrame({'Game Title': game_name,'Critic Score': critic_avg})
    Top10Game_user_score['User Score'] = pd.to_numeric(Top10Game_user_score['User Score'],errors='coerce')
    Top10Game_critic_avg['Critic Score'] = pd.to_numeric(Top10Game_critic_avg['Critic Score'],errors='coerce')
    
    if pd.DataFrame({'game_name': game_name,'user_score': user_score,'critic_avg': critic_avg}).empty:
        print("Query Failed! Please retype! ")
        time.sleep(1)

    else:
        IPython.display.clear_output()
        print("Top 10 Games by Critic Scores")
        display(Top10Game_critic_avg
               .sort_values(by=['Critic Score'],axis=0, ascending=False, inplace=False, na_position='last')
                            .reset_index(drop=True).set_index(Top10Game_critic_avg.index+1).head(10)
                            .style
                            .highlight_between(left=80.0,right=100.0,subset=['Critic Score'],props='color:white;background-color:rgb(102,204,51);font-weight:bold')
                            .highlight_between(left=40.0,right=79.9,subset=['Critic Score'],props='color:white;background-color:rgb(255,204,51);font-weight:bold')
                            .highlight_between(left=0.0,right=39.9,subset=['Critic Score'],props='color:white;background-color:rgb(255,0,0);font-weight:bold')
                            .format(format_dict,na_rep=' -TBD- '))
        print("Top 10 Games by User Ratings")
        display(Top10Game_user_score
               .sort_values(by=['User Score'],axis=0, ascending=False, inplace=False, na_position='last')
                            .reset_index(drop=True).set_index(Top10Game_user_score.index+1).head(10)
                            .style
                            .highlight_between(left=7.5,right=10,subset=['User Score'],props='color:rgb(102,204,51);font-weight:bold')
                            .highlight_between(left=4.0,right=7.5,subset=['User Score'],props='color:rgb(255,204,51);font-weight:bold')
                            .highlight_between(left=0.0,right=3.9,subset=['User Score'],props='color:rgb(255,0,0);font-weight:bold')
                            .format(format_dict,na_rep=' -TBD- '))

    #写入 Data Frame Top 10 平台 第3、4个表

    Top10developer_every_critic_avg= pd.DataFrame({'Game Developer': game_developer,'Critic Score': every_critic_avg})
    Top10developer_every_user_avg= pd.DataFrame({'Game Developer': game_developer,'User Score': every_user_avg})
    Top10developer_every_critic_avg['Critic Score'] = pd.to_numeric(Top10developer_every_critic_avg['Critic Score'],errors='coerce')
    Top10developer_every_user_avg['User Score'] = pd.to_numeric(Top10developer_every_user_avg['User Score'],errors='coerce')
    

    if pd.DataFrame({'game_developer': game_developer,'every_critic_avg': every_critic_avg,'every_user_avg': every_user_avg}).empty:
        print("Query Failed! Please retype! ")
        time.sleep(1)
    else:
        print("Top 10 Developers by Critic Scores")
        display(Top10developer_every_critic_avg.sort_values(by=['Critic Score'],axis=0, ascending=False, inplace=False, na_position='last')
                            .drop_duplicates().reset_index(drop=True).set_index((Top10developer_every_critic_avg.drop_duplicates().reset_index(drop=True).index)+1).head(10)
                            .style
                            .highlight_between(left=80.0,right=100.0,subset=['Critic Score'],props='color:white;background-color:rgb(102,204,51);font-weight:bold')
                            .highlight_between(left=40.0,right=79.9,subset=['Critic Score'],props='color:white;background-color:rgb(255,204,51);font-weight:bold')
                            .highlight_between(left=0.0,right=39.9,subset=['Critic Score'],props='color:white;background-color:rgb(255,0,0);font-weight:bold')
                            .format(format_dict,na_rep=' -TBD- '))
        print("Top 10 Developers by User Ratings")
        display(Top10developer_every_user_avg.sort_values(by=['User Score'],axis=0, ascending=False, inplace=False, na_position='last')
                            .drop_duplicates().reset_index(drop=True).set_index((Top10developer_every_user_avg.drop_duplicates().reset_index(drop=True).index)+1).head(10)
                            .style
                            .highlight_between(left=7.5,right=10,subset=['User Score'],props='color:rgb(102,204,51);font-weight:bold')
                            .highlight_between(left=4.0,right=7.5,subset=['User Score'],props='color:rgb(255,204,51);font-weight:bold')
                            .highlight_between(left=0.0,right=3.9,subset=['User Score'],props='color:rgb(255,0,0);font-weight:bold')
                            .format(format_dict,na_rep=' -TBD- '))

    Top10Game_attractive = pd.DataFrame({'Game Title': game_name,'Popularity': attractives})
    Top10Game_favourite = pd.DataFrame({'Game Title': game_name,'Favorable Rate': favourites})
    Top10Game_favourite['Favorable Rate'] = pd.to_numeric(Top10Game_favourite['Favorable Rate'],errors='coerce')
    Top10Game_attractive['Popularity'] = pd.to_numeric(Top10Game_attractive['Popularity'],errors='coerce')

    if pd.DataFrame({'game_developer': game_developer,'every_critic_avg': every_critic_avg,'every_user_avg': every_user_avg}).empty:
        print("Query Failed! Please retype! ")
        time.sleep(1)
    else:
        print("Top 10 Most Popular Games (by the number of User Rating)")
        display(Top10Game_attractive.sort_values(by=['Popularity'],axis=0, ascending=False, inplace=False, na_position='last')
                            .reset_index(drop=True).set_index(Top10Game_attractive.index+1).head(10)
                            .style
                            .bar(subset=['Popularity'], color='rgb(57,243,187)')
                            .format(format_dict,na_rep=' -TBD- ')
                            )
        print("Top 10 Most Liked Games (by the scale of Positive User Rating)")
        display(Top10Game_favourite.sort_values(by=['Favorable Rate'],axis=0, ascending=False, inplace=False, na_position='last')
                            .reset_index(drop=True).set_index(Top10Game_favourite.index+1).head(10)
                            .style
                            .bar(subset=['Favorable Rate'], color='rgb(255,213,187)')
                            .format(format_dict,na_rep=' -TBD- ')
                )
        print("Wordcloud of Most Popular Genres")
        plot_wordcloud()

In [7]:
def insert_new_game_list(conn):
    count = []
    with conn:
        sqlcount = "SELECT * FROM game_list"
        cur = conn.cursor()
        result_countnum = cur.execute(sqlcount)
        for result in result_countnum:
            count.append(str(result[0]))
        G_ID = (len(count))
        line1 = input("G_ID: ")
        if int(line1) > G_ID:
            line2 = input("number_game_players: ")
            line3 = input("game_publisher: ")
            line4 = input("game_date: ")
            line5 = input("critic_value_all: ")
            line6 = input("game_developer: ")
            line7 = input("game_genres: ")
            line8 = input("game_level: ")
            sqlinsert = " INSERT INTO game_list (G_ID,number_game_players,game_publisher,game_date,critic_value_all,game_developer,game_genres,game_level ) VALUES (?,?, ?, ?, ?, ?, ?, ? )"
            cur.execute(sqlinsert,[line1,line2,line3,line4,line5,line6,line7,line8])
            conn.commit()
            print("Added Succesullfuy!")
        else:
            print("ERROR")
            insert_new_game_list(conn)
    conn.close()

'''insert_new_game_index(conn)'''

def insert_new_game_details(conn):
    count = []
    with conn:
        sqlcount = "SELECT * FROM game_details"
        cur = conn.cursor()
        result_countnum = cur.execute(sqlcount)
        for result in result_countnum:
            count.append(str(result[0]))
        G_ID = (len(count))
        line1 = input("G_ID: ")
        if int(line1) > G_ID:
            line2 = input("game_summary: ")
            sqlinsert = " INSERT INTO game_details (G_ID,game_summary ) VALUES (?,?)"
            cur.execute(sqlinsert,[line1,line2])
            conn.commit()
            print("Added Succesullfuy!")
        else:
            print("ERROR")
            insert_new_game_details(conn)
    conn.close()

def insert_new_game_index(conn):
    count = []
    with conn:
        sqlcount = "SELECT * FROM game_index"
        cur = conn.cursor()
        result_countnum = cur.execute(sqlcount)
        for result in result_countnum:
            count.append(str(result[0]))
        G_ID = (len(count))
        line1 = input("G_ID: ")
        if int(line1) > G_ID:
            line2 = input("game_name: ")
            line3 = input("game_platform: ")
            sqlinsert = " INSERT INTO game_index (G_ID,game_name,game_platform) VALUES (?,?,?)"
            cur.execute(sqlinsert,[line1,line2,line3])
            conn.commit()
            print("Added Succesullfuy!")
        else:
            print("ERROR")
            insert_new_game_index(conn)
    conn.close()


def insert_new_media_views(conn):
    count = []
    with conn:
        line1 = input("G_ID: ")
        line2 = input("critic_value: ")
        line3 = input("critic_grade: ")
        line4 = input("critic_bodys: ")
        sqlinsert = " INSERT INTO media_views (G_ID,critic_value,critic_grade,critic_bodys) VALUES (?,?,?,?)"
        cur = conn.cursor()
        cur.execute(sqlinsert,[line1,line2,line3,line4])
        conn.commit()
        print("Added Succesullfuy!")
    conn.close()

def insert_new_user_rating(conn):
    count = []
    with conn:
        sqlcount = "SELECT * FROM user_rating"
        cur = conn.cursor()
        result_countnum = cur.execute(sqlcount)
        for result in result_countnum:
            count.append(str(result[0]))
        G_ID = (len(count))
        line1 = input("G_ID: ")
        if int(line1) > G_ID:
            line2 = input("user_score: ")
            line3 = input("user_numbers: ")
            line4 = input("user_positive_reviews: ")
            line5 = input("user_mixed_reviews: ")
            line6 = input("user_negative_reviews:  ")
            sqlinsert = " INSERT INTO user_rating (G_ID,user_score,user_numbers,user_positive_reviews,user_mixed_reviews,user_negative_reviews) VALUES (?,?,?,?,?,?)"
            cur.execute(sqlinsert,[line1,line2,line3,line4,line5,line6])
            conn.commit()
            print("Added Succesullfuy!")
        else:
            print("ERROR")
            insert_new_game_index(conn)
    conn.close()

In [8]:
# line = None
MAINFRAME = """
# WORLDWIDE Game Product's Quality in 2020-2022 BASED ON 𝕄𝔼𝕋𝔸ℂℝ𝕀𝕋𝕀ℂ 2020-2022
### IN THIS APPLICATION, YOU WILL BE ABLE TO: ###
***
1. Track information and critic scores of **single game** by **game title** during 2020-2022
2. Track performance of each **game developer** during 2020-2022
3. Track reviews of **specific media** during 2020-2022
4. View **overall performance trend graph** by platform and time period
5. Update the game database
***
"""
    
def mainOptions():
    def returnOptions():
            choice_return = console.input("Do you have other query request? [bold yellow](Y/N)[/]")
            choice_return = choice_return.strip()
            for choose in choice_return:
                if choice_return == 'Y':
                    IPython.display.clear_output()
                    mainOptions()
                elif choice_return == 'N':
                    print("Thank you for using.")
                    print("© AIDM-7360 The Lord of Sham Shui Po") 
                    conn.close()
                    break
                else:
                    print("We are not sure about your choice. Please input again.")
                    continue
                    
    def Pt4_enter(conn):
        print("[1] Query the Top10 Game/Development (By Certain Time Period)")
        print("[2] View Total Trend from 2020 ~ 2022")
        enter = input("Please Choose the option: ")
        if enter == "1":
            Pt4_flatform(conn)
        elif enter == "2":
            alltimetrend(conn)
        else:
            print("error!! Please retype~ ")
            time.sleep(1)
            Pt4_enter(conn)
    
    def Pt5_enter(conn):
        print("[1] Update the game index")
        print("[2] Update the game list")
        print("[3] Update the game details")
        print("[4] Update the media views")
        print("[5] Update the user ratings")
        print("[6] Exit")
        enter = input("Please Choose the option: ")
        if enter == "1":
            insert_new_game_index(conn)
        elif enter == "2":
            insert_new_game_list(conn)
        elif enter == "3":
            insert_new_game_details(conn)
        elif enter == "4":
            insert_new_media_views(conn)
        elif enter == "5":
            insert_new_user_rating(conn)
        elif enter == "6":
            returnOptions()
        else:
            print("error!! Please retype~ ")
            time.sleep(1)
            Pt5_enter(conn)
    
    def showOptions():
        """ Show the options """
        choices = ["Track single game", "Track single game developer","Track single media", "Generate overall trend / Top10 during the exact period","Insert new row into database","exit"]
        print("Please choose following option:")
        print("1. ", choices[0])
        print("2. ", choices[1])
        print("3. ", choices[2])
        print("4. ", choices[3])
        print("5. ", choices[4])
        print("6. ", choices[5])
    conn = sqlite3.connect("Metacritic2020-202211.db")
    showOptions()
    choice = input('And you are going to : ')
    # with conn:
    while True:
        if choice == '6':
            conn.close()
            print("Thank you for using.")
            print("© AIDM-7360 The Lord of Sham Shui Po") 
            break
        elif choice == '1':
            IPython.display.clear_output()
            game_check(conn, line)
            returnOptions()
            break
        elif choice == '2':
            IPython.display.clear_output()
            developersql(conn,line)
            returnOptions()
            break
        elif choice == '3':
            IPython.display.clear_output()
            mediasql(conn,line)
            returnOptions()
            break
        elif choice == '4':
            IPython.display.clear_output()
            Pt4_enter(conn)
            returnOptions()
            break
        elif choice == '5':
            IPython.display.clear_output()
            Pt5_enter(conn)
            returnOptions()
            break
        else:
            print("This option is not available")
            continue                    

In [10]:
conn = sqlite3.connect("Metacritic2020-202211.db")
header=Image(url="https://content.wepik.com/statics/7514372/preview-page0.jpg")
display(header)
mainframe = Markdown(MAINFRAME)
console.print(mainframe)
mainOptions()

And you are going to : 6
